In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Visualización 2 - Bokeh

---

<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

[Imports](#section_imports)

[Dataset](#section_dataset)

[Algunos conceptos básicos](#section_conceptos)

[`circle`](#section_circle)

[`vbar` `circle` `triangle`](#section_vbar_circle_triangle)

[`ColumnDataSource` `gridplot`](#section_columndatasource_gridplot)

[Paletas](#section_paletas)

[`HoverTool`](#section_hover)

[Standalone HTML](#section_output_file)

[Referencias](#section_referencias)

---

<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)

Bokeh es una biblioteca de visualización interactiva para navegadores web. 

Proporciona una construcción elegante y concisa de gráficos versátiles, y ofrece interactividad de alto rendimiento sobre grandes volúmenes de datos o streaming. 

Bokeh provee funcionalidad para hacer dashboards, visualizaciones y aplicaciones de datos de manera rápida y sencilla.

En esta notebook presentamos las herramientas básicas necesarias para construir visualizaciones con Bokeh.


---

<a id="section_imports"></a> 
## Imports 

[volver a TOC](#section_toc)

In [ ]:
import pandas as pd
import numpy as np

---

<a id="section_dataset"></a> 
## Dataset

[volver a TOC](#section_toc)


Bokeh provee algunos "datos de muestra" interesantes en el paquete `bokeh.sampledata`. 

Usaremos en esta notebook datos sintéticos para algunos ejemplos, y los datos históricos de automóviles que se disponibilizan en el dataset autompg.

https://archive.ics.uci.edu/ml/datasets/Auto+MPG

In [ ]:
from bokeh.sampledata.autompg import autompg as data_cars
data_cars.sample(5)

---
<a id="section_conceptos"></a> 
## Algunos conceptos básicos

[volver a TOC](#section_toc)


### Glyphs (glifos)

Son los objetos básicos que conforman los gráficos Bokeh, por ejemplo: lineas, rectángulos, cuadrados. La interface `bokeh.plotting` provee una forma conveniente de crear gráficos en base a glifos.

https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html#userguide-plotting


### Outputs

Hay varias formas de generar salidas de documentos Bokeh. Las más comunes son:

* `output_file`: genera archivos html para las visualizaciones Bokeh 

* `output_notebook`: muestra visualizaciones Bokeh como celdas de jupyter notebooks

Estas funciones generalmente son invocadas junto a las funciones `show` o `save`

Ejemplo:

<code>
    from bokeh.plotting import figure, output_file, show
    output_file("output.html")
    p = figure()
    p.line(x=[1, 2, 3], y=[4,6,2])
    show(p)
</code>


### `bokeh.plotting`

Bokeh provee una interface similar al estilo de Matplotlib. 

Se centra en hacer que los usuarios relacionen los glifos visuales que desean mostrar como representación de sus datos.

La función `figure()` en bokeh.plotting crea un objeto `Figure` que provee métodos para agregar distintos tipos de glifos al gráfico. Y establece ejes por default, grillas, y herramientas sin requerir trabajo extra.

https://docs.bokeh.org/en/latest/docs/reference/plotting.html?highlight=figure#bokeh.plotting.figure.Figure

Ejemplo:

In [ ]:
from bokeh.plotting import figure, output_notebook, show

# create a Figure object
p = figure(plot_width=300, plot_height=300, tools="pan,reset,save")

# add a Circle renderer to this figure
p.circle([1, 2.5, 3, 2], [2, 3, 1, 1.5], radius=0.3, alpha=0.5)

# specify how to output the plot(s)
output_notebook()

# display the figure
show(p)

---

<a id="section_circle"></a> 
## `circle`

[volver a TOC](#section_toc)


Usando el método `circle` de `Figure` vamos a graficar los datos generados

https://docs.bokeh.org/en/latest/docs/reference/plotting.html?highlight=figure#bokeh.plotting.figure.Figure.circle

In [ ]:
x = np.linspace(-15, 15, 200)
y = x*np.cos(x)
print(x)

In [ ]:
p = figure(width=500, height=500)
p.circle(x, y, size=7, color="firebrick", alpha=0.5)
output_notebook()
show(p)

---

<a id="section_vbar_circle_triangle"></a> 
## `vbar` `circle` `triangle`

[volver a TOC](#section_toc)


https://docs.bokeh.org/en/latest/docs/reference/plotting.html?bokeh.plotting.figure.Figure.vbar#bokeh.plotting.figure.Figure.vbar

https://docs.bokeh.org/en/latest/docs/reference/plotting.html?bokeh.plotting.figure.Figure.circle#bokeh.plotting.figure.Figure.circle

https://docs.bokeh.org/en/latest/docs/reference/plotting.html?bokeh.plotting.figure.Figure.triangle#bokeh.plotting.figure.Figure.triangle

Bokeh se basa en la composición de primitivas gráficas que están ligadas a series de datos. 

Calcularemos la media y el desvío estandar del campo `mpg` agrupado por `yr` (model year)

Grafiquemos 

* usando triángulos, los valores de los registros cuyo origen es americano (origin 1)

* usando círculos, los valores de los registros cuyo origen es japones (origin 3)

* usando barras verticales, la media +/- desvío por año



In [ ]:
data_cars_group = data_cars.groupby("yr")
data_cars_agg = data_cars_group["mpg"].agg(['mean', 'std'])
data_cars_agg = data_cars_agg.reset_index()
data_cars_agg.head()

In [ ]:
p = figure(title="MPG Por Año (Japón y USA)")

p.vbar(x = data_cars_agg.yr,
       bottom = data_cars_agg['mean']- data_cars_agg['std'],
       top = data_cars_agg['mean'] + data_cars_agg['std'],
       width = 0.8,
       fill_alpha = 0.2,
       line_color = None, 
       legend_label = "media de mpg +/- 1 desvío")

p.circle(x = data_cars.yr.loc[data_cars.origin==3],
         y = data_cars.mpg.loc[data_cars.origin==3],
         size=10,
         alpha=0.5,
         color="red",
         legend_label="Japanese")

p.triangle(x = data_cars.yr.loc[data_cars.origin==1],
           y = data_cars.mpg.loc[data_cars.origin==1],
           size=10,
           alpha=0.3,
           color="darkblue",
           legend_label="American")

p.legend.location = "top_left"

output_notebook()
show(p)

Observemos que:
    
* Para cada símbolo, estamos graficando un conjunto distinto de datos. Las barras se construyen sobre los datos del DataFrame `data_cars_agg`. Los tríángulos se construyen sobre los datos de DataFrame `data_cars` sólo para los registros con `origin==1`. Los círculos se construyen sobre los datos de DataFrame `data_cars` sólo para los registros con `origin==3`

* Los argmentos x, y reciben la serie de datos a representar. Como alternativa podríamos especificar el valor de `source`, en ese caso x e y tendrían los nombre de las columnas en source que tienen los valores a representar.
    

---

<a id="section_columndatasource_gridplot"></a> 
## `ColumnDataSource` `gridplot`

[volver a TOC](#section_toc)


https://docs.bokeh.org/en/latest/docs/user_guide/data.html?highlight=columndatasource

https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource


**`ColumnDataSource`** es el core de la mayoría de los gráficos de Bokeh y proporciona los datos que visualizamos mediante glifos en el gráfico. 

Con `ColumnDataSource`, es fácil compartir datos entre múltiples gráficos. 

Cuando se utiliza el mismo objeto `ColumnDataSource` como origen de datos de múltiples gráficos, la selección del origen de datos también se comparte. Por lo tanto, es posible utilizar una herramienta de selección para elegir puntos de datos de un gráfico y resaltarlos automáticamente en un segundo gráfico (Selección vinculada).

En el nivel más básico, un objeto `ColumnDataSource` es simplemente un mapeo entre los nombres de columna y las listas de datos. 


**`gridplot`** crea una grilla de plots, y constuye una única toolbar compartida entre todos los plots de la grilla.

https://docs.bokeh.org/en/latest/docs/reference/layouts.html




In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

data_bokeh = ColumnDataSource(data_cars)

options = {'plot_width': 300,
           'plot_height': 300,
           'tools': 'pan, box_select, lasso_select, reset'}

p1 = figure(title="MPG por Año", x_axis_label = "MPG", y_axis_label = "Año", **options)
p1.circle("yr", "mpg", color = "blue", source = data_bokeh)

p2 = figure(title="HP vs. Desplazamiento", x_axis_label = "HP", y_axis_label = "Desplazamiento", **options)
p2.cross("hp", "displ", color = "green", source = data_bokeh)

p3 = figure(title="MPG vs. Desplazamiento", x_axis_label = "MPG", y_axis_label = "Desplazamiento", **options)
p3.diamond("mpg", "displ", size = "cyl", line_color="red", fill_color = None, source = data_bokeh)

p4 = figure(title="Aceleración vs. Desplazamiento", x_axis_label = "Aceleración", y_axis_label = "Desplazamiento",  **options)
p4.triangle("accel", "displ", color = "yellow", source = data_bokeh)

p = gridplot([[p1, p2], [p3, p4]] , toolbar_location="right")

output_notebook()
show(p)

Observemos que:

* el valor del argumento source es el mismo para los cuatro plots generados, y es de tipo `ColumnDataSource`

* aunque definimos el valor de tool para cada uno de los plots, se muestra sólo uno compartido entre todos los plots de la grilla.

* usando cualquiera de las dos herramientas de selección, en cualquiera de los gráficos, estamos seleccionando el mismo conjunto de registros en todos los gráficos.

---

<a id="section_paletas"></a> 
##  Paletas

[volver a TOC](#section_toc)


Bokeh provee una colección de paletas predefinidas que podemos ver en 

https://docs.bokeh.org/en/latest/docs/reference/palettes.html

Usemos cuatro colores de **Category10** para colorear los puntos del gráfico anterior.


In [ ]:
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

data_bokeh = ColumnDataSource(data_cars)

options = {'plot_width': 300,
           'plot_height': 300,
           'tools': 'pan, box_select, lasso_select, reset'}

colors = Category10[4]

p1 = figure(title="MPG por Año", x_axis_label = "MPG", y_axis_label = "Año", **options)
p1.circle("yr", "mpg", color = colors[0], source = data_bokeh)

p2 = figure(title="HP vs. Desplazamiento", x_axis_label = "HP", y_axis_label = "Desplazamiento", **options)
p2.cross("hp", "displ", color = colors[1], source = data_bokeh)

p3 = figure(title="MPG vs. Desplazamiento", x_axis_label = "MPG", y_axis_label = "Desplazamiento", **options)
p3.diamond("mpg", "displ", size = "cyl", line_color=colors[2], fill_color = None, source = data_bokeh)

p4 = figure(title="Aceleración vs. Desplazamiento", x_axis_label = "Aceleración", y_axis_label = "Desplazamiento",  **options)
p4.triangle("accel", "displ", color = colors[3], source = data_bokeh)

p = gridplot([[p1, p2], [p3, p4]] , toolbar_location="right")

output_notebook()
show(p)

---

<a id="section_hover"></a> 
## `HoverTool`

[volver a TOC](#section_toc)


https://docs.bokeh.org/en/latest/docs/user_guide/tools.html

Es una herramienta de inspección pasiva, que por default crea una tabla con información donde cada fila de esa tabla tiene una etiqueta y un valor asociado.

Las etiquetas y los valores se asignan en una lista de tuplas.

Los nombres de campo que comienzan con '$' son "campos especiales". A menudo corresponden a valores que son intrínsecos del gráfico, como las coordenadas del mouse en el espacio de datos o pantalla. Algunos campos especiales se enumeran aquí:

`$index`: indíce del punto seleccionado en el dataset 

`$x`: coordenada x del cursor en el espacio de datos

`$y`: coordenada y del cursor en el espacio de datos

`$sx`: coordenada x del cursor en el espacio de la pantalla (canvas)

`$sy`: coordenada y del cursor en el espacio de la pantalla (canvas)

Los nombres de campo que comienzan con @ están asociados con columnas en el valor del argumento source. 

Por ejemplo, el nombre del campo "@price" mostrará los valores de la columna "precio" cada vez que se active hover para cada registro. 

Ejemplo:

In [ ]:
from bokeh.plotting import ColumnDataSource, figure, output_file, show

#output_file("toolbar.html", mode = 'inline')
output_notebook()

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
))

tools =  ['pan', 'box_zoom', 'reset']

tooltips = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("desc", "@desc"),
]

p = figure(plot_width=400, plot_height=400,            
           tools = tools,
           tooltips=tooltips,
           title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

---

<a id="section_output_file"></a> 
## Standalone HTML

[volver a TOC](#section_toc)


Bokeh también permite guardar plots en archivos HTML. 

Para eso usamos la función `output_file` y `save` (en lugar de `output_notebook()` y `show`)

También podemos usar la combinación `output_file` `show` que abre en una nueva pestaña el gráfico guardado.

Es importante establecer el valor del argumento `mode` que indica cómo se distribuyen los archivos js necesarios para el comportamiento interactivo de la visualización. Los valores posibles se especifican aquí https://docs.bokeh.org/en/latest/docs/reference/resources.html#bokeh.resources.Resources


Guardemos el último gráfico generado, que está asignado a la variable `p` 


In [ ]:
from bokeh.plotting import output_file, save
output_file("test_save_plot.html", mode='inline')
save(p)

Para resetear los settings de output usamos `reset_output`

https://docs.bokeh.org/en/latest/docs/reference/io.html?highlight=reset_output#bokeh.io.output.reset_output

In [ ]:
from bokeh.plotting import reset_output
reset_output()

---

<a id="section_referencias"></a> 
## Referencias

[volver a TOC](#section_toc)


https://docs.bokeh.org/en/latest/index.html

Referencia
https://docs.bokeh.org/en/latest/docs/reference.html#refguide

Galería
https://docs.bokeh.org/en/latest/docs/gallery.html#id3

User Guide
https://docs.bokeh.org/en/latest/docs/user_guide.html#userguide